In [1]:
import os, sys
import torch
import torch.nn as nn
import torch.nn.functional as F

# from models import resnet_ft
# from models import resnet_ft_id
# from models import resnet 
# from models import res2net
# from models import res2net_id
# 基于4-D数据的模型

# Contrastive LOSS 
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        # 0: same   1: diff
        # loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
        #                               (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        
        # 0: diff   1: same
        loss_contrastive = torch.mean((label) * torch.pow(euclidean_distance, 2) +
                                      (1-label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        # loss里的euclidean_distance 和 margin可能有问题，导致过拟合
        # loss_contrastive = torch.sum((label) * torch.pow(euclidean_distance, 2) +
        #                               (1-label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        return loss_contrastive

In [5]:
class LSTM_Net(nn.Module):
    # this is the baseline for code test
    def __init__(self):
        """
        Arguments:
            nn {[type]} -- [description]
            input batch_data: size = [batch_size, 1, 61, 73, 61]
        """
        super(LSTM_Net, self).__init__()
        self.base = nn.Sequential(
            nn.AvgPool3d(3),
            nn.Conv3d(in_channels=1,out_channels=4,kernel_size=(3,3,3),
                 stride=1,padding=0,dilation=1,bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(4),
            
            nn.Conv3d(in_channels=4,out_channels=8,kernel_size=(3,3,3),
                 stride=1,padding=0,dilation=1,groups=1,bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(8),

            nn.Conv3d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(8)
        )
        
        self.gap = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(960, 128)
        self.lstm = nn.LSTM(128,16)
        
    def forward(self, x):
        output=[]
        for i in range(x.shape[5])
            y = self.base(x)
            y = self.gap(y)
            y = output.view(y.size()[0], -1)
            y = self.fc(y)
            output.append(y)
        return output_f

In [6]:
import os
import nilearn as nil
from nilearn.image import new_img_like, load_img
from nilearn import image,plotting

import numpy as np

D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
ab_dir = 'e:/master_project/fMRI/data/siam/train_bold'
bold_list = os.listdir(ab_dir)
b = load_img(os.path.join(ab_dir,bold_list[0]))
print(b.shape)
data = b.get_fdata()

(49, 58, 47, 232)


In [19]:
model = LSTM_Net()
inputs = torch.randn(8,1,49,58,47,232)
p1 = inputs[:,:,:,:,:,0:16]



[tensor([[ 2.4770, -1.3375,  0.8278]]),
 tensor([[-0.1497,  0.0024, -2.1038]]),
 tensor([[-1.0671,  0.0348, -2.0075]]),
 tensor([[-0.7579, -0.0211, -0.2151]]),
 tensor([[ 1.6865, -0.9434,  0.7096]])]